In [25]:
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [26]:
def read_data(filepath):
    df = pd.read_csv(filepath, index_col=0)
    return df


In [27]:
filepath_min_max_1002 = "../database/features/1002_min_max.csv"
filepath_std_1002 = "../database/features/1002_stand_norm.csv"
filepath_oryg_1202 = "../database/features/1202.csv"
filepath_std_new_1202 = "../database/features/1202_stand_norm.csv"
filepath_std_new_1402 = "../database/features/1402_stand_norm_with_loudness_norm.csv"

target_dict = {'happy': 0, 'angry': 1, 'sad': 2, 'relaxed': 3}

In [28]:
df = read_data(filepath_std_new_1402)
df.replace({"emotion": target_dict}, inplace=True)

# columns: 
cent_mean,cent_std,cent_skew,rolloff_mean,rolloff_std,rolloff_skew,flux_mean,flux_std,flux_skew,
flatness_mean,flatness_std,flatness_skew,
mfccs_mean_0,mfccs_mean_1,mfccs_mean_2,mfccs_mean_3,mfccs_mean_4,mfccs_mean_5,mfccs_mean_6,mfccs_mean_7,mfccs_mean_8,
mfccs_mean_9,mfccs_mean_10,mfccs_mean_11,mfccs_mean_12,mfccs_mean_13,mfccs_mean_14,mfccs_mean_15,mfccs_mean_16,
mfccs_mean_17,mfccs_mean_18,mfccs_mean_19,mfccs_std_0,mfccs_std_1,mfccs_std_2,mfccs_std_3,mfccs_std_4,mfccs_std_5,
mfccs_std_6,mfccs_std_7,mfccs_std_8,mfccs_std_9,mfccs_std_10,mfccs_std_11,mfccs_std_12,mfccs_std_13,mfccs_std_14,
mfccs_std_15,mfccs_std_16,mfccs_std_17,mfccs_std_18,mfccs_std_19,contrast_mean_0,contrast_mean_1,contrast_mean_2,
contrast_mean_3,contrast_mean_4,contrast_mean_5,contrast_mean_6,contrast_std_0,contrast_std_1,contrast_std_2,contrast_std_3,
contrast_std_4,contrast_std_5,contrast_std_6,#chroma_mean_0,chroma_mean_1,chroma_mean_2,chroma_mean_3,chroma_mean_4,
chroma_mean_5,chroma_mean_6,chroma_mean_7,chroma_mean_8,chroma_mean_9,chroma_mean_10,chroma_mean_11,chroma_std_0,
chroma_std_1,chroma_std_2,chroma_std_3,chroma_std_4,chroma_std_5,chroma_std_6,chroma_std_7,chroma_std_8,chroma_std_9,
chroma_std_10,chroma_std_11,emotion

As stated in the article best results give stacked MFCC & Spec. Contrast features

In [29]:
for i, col in enumerate(df.columns):
    print(i, col)

0 cent_mean
1 cent_std
2 cent_skew
3 rolloff_mean
4 rolloff_std
5 rolloff_skew
6 flux_mean
7 flux_std
8 flux_skew
9 flatness_mean
10 flatness_std
11 flatness_skew
12 mfccs_mean_0
13 mfccs_mean_1
14 mfccs_mean_2
15 mfccs_mean_3
16 mfccs_mean_4
17 mfccs_mean_5
18 mfccs_mean_6
19 mfccs_mean_7
20 mfccs_mean_8
21 mfccs_mean_9
22 mfccs_mean_10
23 mfccs_mean_11
24 mfccs_mean_12
25 mfccs_std_0
26 mfccs_std_1
27 mfccs_std_2
28 mfccs_std_3
29 mfccs_std_4
30 mfccs_std_5
31 mfccs_std_6
32 mfccs_std_7
33 mfccs_std_8
34 mfccs_std_9
35 mfccs_std_10
36 mfccs_std_11
37 mfccs_std_12
38 mfccs_mean_1_0
39 mfccs_mean_1_1
40 mfccs_mean_1_2
41 mfccs_mean_1_3
42 mfccs_mean_1_4
43 mfccs_mean_1_5
44 mfccs_mean_1_6
45 mfccs_mean_1_7
46 mfccs_mean_1_8
47 mfccs_mean_1_9
48 mfccs_mean_1_10
49 mfccs_mean_1_11
50 mfccs_mean_1_12
51 mfccs_std_1_0
52 mfccs_std_1_1
53 mfccs_std_1_2
54 mfccs_std_1_3
55 mfccs_std_1_4
56 mfccs_std_1_5
57 mfccs_std_1_6
58 mfccs_std_1_7
59 mfccs_std_1_8
60 mfccs_std_1_9
61 mfccs_std_1_10
62 

#### Regular:
From 12 till 51 - mfcc features (12-31 mean, 32-51 std)

From 52 till 65 - contrast features (52-58 mean, 59-65)

#### Additional mfcc:
From 12 till 89 - mfcc features (12-37 mean and std for 22ms, 38-63 mean and std for 1s, 64-80 mean and std for 30s)

From 90 till 103 - contrast features (90-96 mean, 97-103 std)

In [30]:
df_all_X = df.iloc[:, :-1]

df_X_only_mean_0 = df.iloc[:, 12:25]
df_X_only_mean_1 = df.iloc[:, 38:51]
df_X_only_mean_2 = df.iloc[:, 65:77]
df_X_only_mean = pd.concat([df_X_only_mean_0, df_X_only_mean_1, df_X_only_mean_2], axis=1)

df_X_mean_with_chroma = pd.concat([df_X_only_mean, df_all_X.iloc[:, 104:128]])

df_X_mean_with_others = pd.concat([df_X_only_mean, df_all_X.iloc[:, 0:12]], axis=1)

df_X_10_most_imp = pd.concat([df_all_X.iloc[0:23], df_all_X.iloc[25:36], df_all_X.iloc[38:49], 
                              df_all_X.iloc[51:62], df_all_X.iloc[64:75], df_all_X.iloc[77:88], df_all_X.iloc[90:115],
                              df_all_X.iloc[116:127]])

to_check = [df_all_X, df_X_only_mean, df_X_mean_with_chroma, df_X_mean_with_others, df_X_10_most_imp]

In [31]:
df_all_X

,cent_mean,cent_std,cent_skew,rolloff_mean,rolloff_std,rolloff_skew,flux_mean,flux_std,flux_skew,flatness_mean,...,chroma_std_2,chroma_std_3,chroma_std_4,chroma_std_5,chroma_std_6,chroma_std_7,chroma_std_8,chroma_std_9,chroma_std_10,chroma_std_11
0,0.886235,2.661419,-0.392094,1.003584,1.709682,-1.047356,3.058107,3.422221,-0.813941,1.280322,...,0.774619,1.281111,0.207867,0.847959,-0.512251,-0.314443,0.475179,0.350780,1.909263,0.023470
1,1.076638,0.750903,-0.668407,1.230839,0.288231,-0.909419,1.057834,0.860447,-0.689943,0.041510,...,-1.440889,-0.349095,2.020883,0.387564,1.303361,-0.936628,1.174089,-1.312649,-1.575754,1.215836
2,-0.135493,0.842640,-0.585928,-0.132488,0.375929,-0.611548,0.175246,0.763973,-0.163602,1.466483,...,1.706911,0.521969,0.960311,0.987553,0.348672,0.291102,0.384410,0.457963,0.511680,0.818302
3,0.394643,-0.779722,-0.662729,0.337390,-0.833830,-0.271953,-0.385620,-0.285913,0.294481,-0.224906,...,-1.827822,0.643682,0.111901,-0.444991,0.718525,-0.745006,0.884443,-0.451011,-0.167487,0.663820
4,1.287725,0.091719,0.208139,1.333860,-0.766094,-0.499379,0.241742,0.096354,-0.597113,0.704652,...,0.065256,-0.178437,0.125179,-0.102301,0.109225,0.681602,-0.401027,0.030300,0.107247,0.322578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,-0.534440,-0.386203,0.648437,-0.426655,-0.006699,0.160528,-0.589115,-0.557446,-0.416250,-0.575899,...,0.728976,-0.018372,0.030034,0.800774,-0.608735,0.649728,0.468678,0.474217,-0.112463,0.069886
1397,-0.062259,0.824486,-0.017273,0.055609,1.339042,-0.042017,0.768797,0.471167,-0.451579,-0.473011,...,0.166642,2.166625,-0.027258,-0.019160,0.066823,-0.910992,1.688474,-1.195852,0.664532,0.910804
1398,0.322280,-0.719337,-1.889887,0.271432,-0.520580,-0.893334,-0.901854,-1.339645,-0.906422,0.847095,...,-0.496289,-0.059592,-0.458273,-0.464030,0.448779,-0.026821,0.111134,0.553548,0.474813,1.701667
1399,1.267988,0.777335,0.290388,1.336414,-0.403297,-0.563884,-0.172805,-0.423238,-0.485957,0.027631,...,-0.195888,0.081963,1.159354,-0.204841,0.552907,0.798436,0.768069,-1.871702,-1.380916,0.923622


In [32]:
df_all_Y = df.iloc[:,-1]

In [33]:
df_all_Y

0       0
1       1
2       2
3       0
4       1
       ..
1396    1
1397    3
1398    3
1399    3
1400    3
Name: emotion, Length: 1401, dtype: int64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(df_all_X, df_all_Y, test_size=0.3,random_state=45)

In [35]:
X_train

,cent_mean,cent_std,cent_skew,rolloff_mean,rolloff_std,rolloff_skew,flux_mean,flux_std,flux_skew,flatness_mean,...,chroma_std_2,chroma_std_3,chroma_std_4,chroma_std_5,chroma_std_6,chroma_std_7,chroma_std_8,chroma_std_9,chroma_std_10,chroma_std_11
1173,0.085784,-1.096502,-1.485410,0.179456,-1.256909,-1.245597,-0.723449,-0.920401,0.369485,0.025234,...,-0.499355,0.081429,-1.101415,0.075466,-0.287639,-1.169462,-0.562163,-1.021403,0.553746,-0.177888
200,0.072980,1.299642,-0.977132,0.245041,1.253299,-0.844082,-0.549579,-0.080956,0.840069,-0.098833,...,-1.334145,-1.881075,-0.496928,-1.174672,-0.077100,-0.472965,-1.021179,0.984785,-0.671354,0.140515
1310,1.195316,1.005607,-0.372412,0.828445,-0.270627,-0.409948,1.431399,0.732256,-1.068100,0.000237,...,-0.725583,0.876059,-0.581013,0.651271,-1.035785,-0.709956,1.401900,-0.288397,0.276021,-0.409152
137,-0.534935,1.339240,0.888377,-0.552208,0.866288,0.441930,-0.257834,0.029937,2.148712,-0.449174,...,0.652368,0.124236,-1.187859,0.444133,-0.393761,1.787663,-0.814263,-0.210042,0.690604,-1.456854
206,-2.277369,-0.999130,2.572518,-2.313855,-0.517508,2.481921,-0.752701,-0.202658,-0.049555,-0.843378,...,0.867343,-1.211529,2.564081,-0.987006,-0.397469,0.919829,-1.293324,-0.300575,-1.378401,1.428660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,-0.050635,-0.559567,0.017983,0.207507,-0.216801,-0.091363,-0.404352,-0.212827,0.468253,-0.382994,...,0.081428,-0.747468,0.253890,-1.428515,1.472663,-1.331640,0.612302,0.533338,-1.713121,-0.108147
163,1.020670,0.388717,-0.983153,1.161059,0.345447,-1.171769,0.168377,0.361988,-0.238911,0.136714,...,1.395541,-0.486277,0.581325,-0.131808,1.086920,-0.251100,-0.723084,0.876214,-0.945702,-0.674431
607,1.632582,3.083537,-1.025798,1.545860,1.945258,-1.462939,0.455779,0.365010,-0.287338,2.210915,...,0.430228,0.078650,1.507204,0.586537,0.886175,-0.151379,0.746172,1.759622,0.263905,0.343452
414,-0.935170,0.685340,0.887736,-0.855878,0.821503,0.646606,1.165124,0.820176,-0.917878,0.004102,...,1.411163,0.439845,1.644541,-0.193315,-0.592418,0.822714,-0.228782,2.078777,-0.710564,-1.045660


In [36]:
print(y_train[y_train == 0].count(),
y_train[y_train == 1].count(),
y_train[y_train == 2].count(),
y_train[y_train == 3].count())

235 247 251 247


## Multiclass SVM

In [38]:
for j,i in enumerate(to_check):
    C_range = np.logspace(-2, 10, 13)
    gamma_range = np.logspace(-9, 3, 13)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
    grid.fit(i, df_all_Y)

    print(
        "The best parameters are %s with a score of %0.2f"
        % (grid.best_params_, grid.best_score_)
    )

The best parameters are {'C': 10000000.0, 'gamma': 1e-09} with a score of 0.27


KeyboardInterrupt: 

In [ ]:
X_custom_columns = df.columns[:90]
X_custom_data = pd.Series([2018.000135431536,779.8711755595874,0.725172770877271,4359.963102176539,1897.676502582215,0.363897161791218,1.2039237,1.1325041,3.0907743618420787,0.027259167,0.10077373,9.074147521171854,-62.753525,101.83419,0.06773582,17.334158,15.144818,14.202075,1.5652041,-0.75855285,-4.5298147,-1.120752,-8.084144,5.967767,-4.2661266,2.2056193,0.28677782,2.7689784,-4.533309,3.3648374,-3.727215,-1.9008431,114.546165,30.451164,21.641888,12.46622,11.214978,15.098663,9.912447,8.80398,7.603675,7.861005,8.565976,9.284278,6.8444614,6.9627647,6.2478733,6.0670047,5.9645305,6.184176,5.9584875,5.1812186,18.716550791761744,14.929868241054823,19.18835983874837,17.558404307835872,19.600366806491284,17.249195434400132,46.067023270469946,5.401574150126814,4.131021405085011,4.858880177542373,3.322288314784178,4.21941867858195,2.7868359930619984,4.657343543838246,0.3793038,0.34309706,0.473007,0.41531375,0.5669585,0.38024753,0.28839794,0.4387851,0.32310018,0.39071566,0.28945285,0.43881184,0.28005856,0.24435385,0.33295026,0.2545338,0.33695054,0.25274646,0.21735792,0.3533407,0.21321927,0.32738724,0.19917777,0.2855035])
y_custom_columns = df.columns[90]
y_custom_data = pd.Series([0])
X_custom_new = pd.DataFrame(X_custom_data.to_frame().T, columns=X_custom_columns)
y_custom_new = pd.DataFrame(data=[0], columns = [y_custom_columns])

In [32]:
svm_clf = svm.SVC(kernel='rbf',gamma=0.000001, C=1000, decision_function_shape="ovr")
svm_clf.fit(X_train, y_train)

y_pred = svm_clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.25761772853185594


Accuracy for std, 1202 records, kernel='rbf',gamma=0.0000001, C=10000000000, decision_function_shape="ovr": 27.57%

Accuracy for std, 1402 records, the best parameters are {'C': 1000.0, 'gamma': 1e-06} with a score of 0.28, just mfcc and contrast
